*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers



In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-02-27 12:04:12--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.3’

train-data.tsv.3    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-02-27 12:04:13 (10.5 MB/s) - ‘train-data.tsv.3’ saved [358233/358233]

--2022-02-27 12:04:13--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.3’

valid-data.tsv.3    100%[========

In [ ]:
train= pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])


In [ ]:
test= pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])


In [ ]:
print(len(train))

4179


In [ ]:
train_label=pd.factorize(train['type'])
train.pop('type')

0        ham
1        ham
2        ham
3        ham
4        ham
        ... 
4174     ham
4175     ham
4176    spam
4177    spam
4178     ham
Name: type, Length: 4179, dtype: object

In [ ]:
test_label=pd.factorize(test['type'])
test.pop('type')

0        ham
1        ham
2        ham
3        ham
4        ham
        ... 
1387     ham
1388     ham
1389     ham
1390    spam
1391    spam
Name: type, Length: 1392, dtype: object

In [ ]:
#train_label=np.array(list(map(lambda x: 1 if x=="ham" else 0, train_label)))
print(train_label)

(array([0, 0, 0, ..., 1, 1, 0]), Index(['ham', 'spam'], dtype='object'))


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(train['msg'])

traintt = (tokenizer.texts_to_sequences(train['msg']))

testtt = ((tokenizer.texts_to_sequences(test['msg'])))


In [ ]:
from keras.preprocessing import sequence
train_inp= sequence.pad_sequences(traintt, maxlen=1000)


In [ ]:
model2 = keras.Sequential()

#model2.add(tf.keras.layers.Input(shape=[1000]))
model2.add(tf.keras.layers.Embedding(1000, 50, input_length=1000))
model2.add(tf.keras.layers.LSTM(64))
model2.add(tf.keras.layers.Dense(256, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.5))
model2.add(tf.keras.layers.Dense(1, activation='relu'))

model2.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 1000, 50)          50000     
                                                                 
 lstm_9 (LSTM)               (None, 64)                29440     
                                                                 
 dense_18 (Dense)            (None, 256)               16640     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 257       
                                                                 
Total params: 96,337
Trainable params: 96,337
Non-trainable params: 0
_________________________________________________________________


In [ ]:
r = model2.fit(train_inp, train_label[0],
              batch_size=128, epochs=10)

Epoch 1/10
33/33 [==============================] - 48s 1s/step - loss: 0.2837 - accuracy: 0.9000
Epoch 2/10
33/33 [==============================] - 44s 1s/step - loss: 0.0802 - accuracy: 0.9816
Epoch 3/10
33/33 [==============================] - 45s 1s/step - loss: 0.0567 - accuracy: 0.9885
Epoch 4/10
33/33 [==============================] - 44s 1s/step - loss: 0.0481 - accuracy: 0.9919
Epoch 5/10
33/33 [==============================] - 45s 1s/step - loss: 0.0534 - accuracy: 0.9933
Epoch 6/10
33/33 [==============================] - 45s 1s/step - loss: 0.0397 - accuracy: 0.9950
Epoch 7/10
33/33 [==============================] - 45s 1s/step - loss: 0.0341 - accuracy: 0.9969
Epoch 8/10
33/33 [==============================] - 46s 1s/step - loss: 0.0334 - accuracy: 0.9966
Epoch 9/10
33/33 [==============================] - 47s 1s/step - loss: 0.0318 - accuracy: 0.9974
Epoch 10/10
33/33 [==============================] - 47s 1s/step - loss: 0.0330 - accuracy: 0.9969


In [ ]:
def padding(X):
  
  x = tokenizer.texts_to_sequences(X)
  return sequence.pad_sequences(x, maxlen=1000)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
from sklearn.ensemble import RandomForestClassifier 


def predict_message(pred_text):
    out = model.predict(padding(pd.Series([pred_text])))[0]

    return (out[0], ("ham" if out<0.5 else "spam"))



pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

(0.0, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
